In [11]:
import os
import sys
app_path = os.path.abspath('..')
sys.path.insert(0, app_path)

from dotenv import load_dotenv

from app.telegram_client import get_client_with_auth
from app.chroma_client import get_embeddings, get_client, load_data_to_chroma


load_dotenv()

CHANNEL_NAME = os.getenv('TELEGRAM_CHAT_NAME')
CHANNEL_ID = os.getenv('TELEGRAM_CHAT_ID')

In [12]:
telegram_client = await get_client_with_auth()

In [13]:
from typing import Any
from tqdm import tqdm
from telethon.tl.types import PeerChannel

CHAT_MESSAGES_LIMIT = 1000

async def get_messages_from_channel(
    telegram_client, channel_id: int | str, min_id: int = 0, max_id: int = 0, chunks: int = 10
) -> dict[int, dict[str, Any]]:
    messages_data = {}
    if channel_id < 0:
        channel_id = PeerChannel(channel_id)

    channel = await telegram_client.get_entity(channel_id)
    last_seen_message_id = max_id

    for _i in tqdm(range(chunks), desc="Fetching chunks", position=0, leave=False, colour='green'):
        if all((last_seen_message_id, min_id)) and last_seen_message_id == min_id:
            print("Reached last unseen message")
            break

        async for message in telegram_client.iter_messages(
            channel, min_id=min_id, offset_id=last_seen_message_id, limit=CHAT_MESSAGES_LIMIT
        ):
            if not message.text or len(message.text) < 10:
                continue

            message_text = message.text
            if message.is_reply:
                if message.reply_to_msg_id not in messages_data:
                    original_message = await message.get_reply_message()
                    if original_message:
                        messages_data[original_message.id] = {
                            'text': original_message.text,
                            'metadata': {
                                'sender_name': original_message.sender.username or '',
                                'id': original_message.id,
                                'date_str': original_message.date.isoformat(),
                                'date': original_message.date.timestamp(),
                            }
                        }

                original_message = messages_data.get(message.reply_to_msg_id)
                if original_message:
                    message_text = f'>> {original_message['text']}\n\n {message.text}'
                else:
                    message_text = f'>> [[ORIGINAL MESSAGE REMOVED]]\n\n {message.text}'

            messages_data[message.id] = {
                'text': message_text,
                'metadata': {
                    'sender_name': message.sender.username or '',
                    'id': message.id,
                    'date': message.date.timestamp(),
                }
            }

        last_seen_message_id = message.id
        if last_seen_message_id == 0:
            break

    return messages_data

In [14]:
# ru_model_name = "cointegrated/rubert-tiny2"
ru_model_name = "intfloat/multilingual-e5-large"
embeddings = get_embeddings(ru_model_name)

chroma_client_from_telegram = get_client(f'telegram_{CHANNEL_NAME}', embeddings)

OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB. GPU 0 has a total capacity of 8.00 GiB of which 4.35 GiB is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 2.49 GiB is allocated by PyTorch, and 47.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [6]:
from langchain_chroma import Chroma
from datetime import datetime, timedelta, UTC

def get_last_seen_id_from_db(chroma_client: Chroma):
    doc_dict = chroma_client.get(
        where={"date": {"$gte": (datetime.now(UTC) - timedelta(days=30)).timestamp()}},
        include=['metadatas'],
    )
    if not doc_dict['metadatas']:
        return 0

    return max(meta['id'] for meta in doc_dict['metadatas'])

def get_earliest_message_id(chroma_client: Chroma):
    doc_dict = chroma_client.get(include=['metadatas'])
    if not doc_dict['metadatas']:
        return 0

    return min(meta['id'] for meta in doc_dict['metadatas'])

async def get_early_messages_from_chat(channel_id, telegram_client, chroma_client: Chroma, chunks=10):
    earliest_message_id = get_earliest_message_id(chroma_client)
    print("Earliest seen id:", earliest_message_id)
    messages = await get_messages_from_channel(
        telegram_client, channel_id, max_id=earliest_message_id, chunks=chunks
    )
    print("Got messages:", len(messages))
    return messages

async def get_last_messages_from_chat(channel_id, telegram_client, chroma_client: Chroma, chunks=10):
    last_seen_id = get_last_seen_id_from_db(chroma_client)
    print("Last seen id:", last_seen_id)
    messages = await get_messages_from_channel(telegram_client, channel_id, min_id=last_seen_id, chunks=chunks)
    print("Got messages:", len(messages))
    return messages

In [1]:
new_messages = await get_last_messages_from_chat(
    channel_id=CHANNEL_ID,
    telegram_client=telegram_client,
    chroma_client=chroma_client_from_telegram,
    chunks=10
)

NameError: name 'get_last_messages_from_chat' is not defined

In [30]:
new_messages = await get_early_messages_from_chat(
    channel_id=CHANNEL_ID,
    telegram_client=telegram_client,
    chroma_client=chroma_client_from_telegram,
    chunks=10
)

Earliest seen id: 80265


                                                                2.22s/it]

Got messages: 9141


In [9]:
print(len(new_messages))
print(list(new_messages.values())[-1:])

9279
[{'text': 'Привет! Может, кто-то собирается в Беларусь в течение месяца может захватить пакет 5 кг?', 'metadata': {'sender_name': 'elli_l_elli', 'id': 104544, 'date': 1713462583.0}}]


In [32]:
load_data_to_chroma(chroma_client_from_telegram, new_messages.values(), reset=False)

Loaded 9089 documents into ChromaDB.


In [11]:
from langchain import PromptTemplate
from langchain_ollama import ChatOllama


llm_qwen3_8b = ChatOllama(
    model="qwen3:8b",
)

In [35]:
user_query = "Выведи самые токсичные сообщения в чате."
user_query = "Выведи самые позитивные сообщения в чате."
user_query = "Можно ли компенсировать отель или аквапарк через велнес БТА или BTA?"
user_query = "Как получить детские деньги, компенсацию по уходу за ребенком."

In [24]:
ru_telegram_prompt_expanding = PromptTemplate(
    input_variables=["user_query", "n"],
    template="""
    Напиши {n} гипотетических ответов на запрос пользователя.
    Не добавляй заголовков, авторов и другой дополнительной информации к твоим ответам.

    Вопрос пользователя:
    {user_query}

    Гипотетические ответы:
    """,
)

hyde_chain_r1_8b = ru_telegram_prompt_expanding | llm_qwen3_8b
raw_hypothetical_answers = hyde_chain_r1_8b.invoke({"user_query": user_query, "n": 15}).content
raw_hypothetical_answers

'<think>\nХорошо, пользователь просит вывести самые позитивные сообщения в чате. Мне нужно придумать 15 гипотетических ответов. Сначала подумаю, что такое позитивные сообщения. Это, скорее всего, сообщения, которые выражают радость, вдохновение, поддержку, благодарность и т.д. Надо убедиться, что каждый ответ уникален и отражает разные аспекты позитива.\n\nМожет, начать с примеров конкретных фраз. Например, "Пусть сегодня будет отличным днем!" или "Ты молодец, что нашел это!" Это звучит позитивно. Нужно разнообразить структуру предложений, чтобы не было повторений. Возможно, использовать разные эмоции: вдохновение, надежда, благодарность, поддержка.\n\nВажно избегать общих фраз и сделать каждый ответ немного разным. Например, можно добавить что-то про достижения, природу, позитивные изменения. Также стоит учесть разные контексты: личные, профессиональные, общие. Например, "Все будет хорошо, верю в тебя!" или "Ты уже сделал огромный шаг вперед!".\n\nНужно проверить, чтобы ответы не были

In [25]:
user_query = raw_hypothetical_answers.split('</think>', maxsplit=1)[-1]

In [36]:
related_docs = chroma_client_from_telegram.similarity_search_with_relevance_scores(user_query, k=15)
related_docs

[(Document(id='685dd014-4a90-4aef-aaf0-88c0899b36ae', metadata={'date': 1666881304.0, 'id': 72554, 'sender_name': 'vasserling'}, page_content='Подскажите, что нажимать для получения пособия на детей?'),
  0.8435551130124125),
 (Document(id='1e9d56c6-f056-4bf0-aaf3-a2d966ebdb59', metadata={'date': 1677788963.0, 'id': 82605, 'sender_name': 'vasserling'}, page_content='Подскажите, можно ли (и как) воспользоваться медстраховкой для ребенка?'),
  0.8166180927515844),
 (Document(id='8e0dbac4-2079-4b2d-a901-d80da8c4b12f', metadata={'date': 1666860564.0, 'id': 72469, 'sender_name': 'radroxx'}, page_content='>> подать заявку на компенсацию 80 евро на детей\n\n И если ЗП позволяет можно податься на финансовую помощь от государства'),
  0.8132342908720387),
 (Document(id='1f0bedc7-0fdd-4542-9a80-c0348db8e9ef', metadata={'date': 1724317551.0, 'id': 108634, 'sender_name': 'sharewax'}, page_content='счёт-фактуру просишь и отправляешь в компенсацию'),
  0.8113701568358261),
 (Document(id='f427a233-b5

In [23]:

from datetime import datetime, timedelta, UTC

related_docs = chroma_client_from_telegram.get(
    where={"date": {"$gte": (datetime.now(UTC) - timedelta(days=30)).timestamp()}},
)
meta_docs = list(zip(related_docs['metadatas'], related_docs['documents']))

context = "\n\n---\n\n".join(f"{meta['date']} - {meta['sender_name']}: {doc}" for meta, doc in meta_docs)

In [25]:
len(related_docs['metadatas'])

805

In [37]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, related_docs)

t_link_base = "https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D"

print("\n\n---\n\n".join(f"""
{doc.metadata['date']} {t_link_base}{doc.metadata['id']} - {doc.page_content}
""" for doc, _score in filtered_related_docs))


1666881304.0 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D72554 - Подскажите, что нажимать для получения пособия на детей?


---


1677788963.0 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D82605 - Подскажите, можно ли (и как) воспользоваться медстраховкой для ребенка?


---


1666860564.0 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D72469 - >> подать заявку на компенсацию 80 евро на детей

 И если ЗП позволяет можно податься на финансовую помощь от государства


---


1724317551.0 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D108634 - счёт-фактуру просишь и отправляешь в компенсацию


---


1718264314.0 https://web.telegram.org/k/#?tgaddr=tg%3A%2F%2Fprivatepost%3Fchannel%3D1133953167%26post%3D106511 - У кого дети ходят в гос детский садик - сколько денег вы платите в месяц?


---


1675428122.0 https://we

In [10]:
filtered_related_docs = filter(lambda doc_score: doc_score[-1] > 0.3, related_docs)

context = "\n\n---\n\n".join(f"{doc.metadata['date']} - {doc.page_content}" for doc, _score in filtered_related_docs)

In [11]:
# user_query = "Здесь собраны последние сообщения из чата. Расскажи про основные темы и выводы."

ru_telegram_prompt = PromptTemplate(
    input_variables=["context", "user_query"],
    template="""
    Ты полезный AI ассистент, который отвечает на вопросы пользователя на основе контекста.
    Контекст это релевантные вопросу сообщения из телеграм чата.

    Контекст:
    {context}

    Вопрос пользователя:
    {user_query}

    Если в контексте недостаточно информации, чтобы ответить на вопрос пользователя, то скажи, что недостаточно информации.

    Ответ:
    """,
)

hyde_chain_r1_8b = ru_telegram_prompt | llm_qwen3_8b
llm_response = hyde_chain_r1_8b.invoke({"user_query": user_query, "context": context}).content
print(llm_response)

<think>
Okay, let's tackle this query. The user is asking for the best banyas or saunas. Looking at the context, there are a few mentions. First, someone suggested a place in Tarkai with a cabin on a lake, but it's an electric heater, so not great for heat. Then there's HBH Vilnius with 3 cottages that have saunas, but it's unclear if you can bring your own food. Another user mentioned Fitus.lt as a gym with a pool, which is expensive but has good facilities. There's also mention of Forum Palace for Thai boxing with a Belarusian trainer, but that's more about fitness than saunas. The user also asked about a 25-50m pool, but that's a different context.

So, the main options are the Tarkai cabin and HBH cottages. The Tarkai one is nice but the heater isn't great. HBH has saunas but food policy is uncertain. The user wants a place for 10 people and to bring their own food. The Tarkai cabin might work if the heater is okay, but the HBH cottages might be better if they allow food. However, 